<a href="https://colab.research.google.com/github/Sona-12/PhD_Tasks/blob/main/p53_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import scipy.integrate
import scipy.io
import pylab as plt

## Setting parameters

In [ ]:
est_params = pd.read_csv('p53_parameters_summaryTable.txt',sep=' ',index_col=False)
est_params = est_params.set_index('objFun')
est_params

,rSeed,kpatm_dox,kpatm_rad,jpatm_dox,jpatm_rad,kdpatm_dox,kdpatm_rad,jdpatm_dox,jdpatm_rad,kpchk2,...,jsmdm21,kswip11,jswip11,kdp53,kdp53a,jup53,jup53s,juhipk2,sf_s15,sf_s46
objFun,,,,,,,,,,,,,,,,,,,,,
6.942752,98,1.192694,0.714128,0.999984,0.010015,0.378553,4.287005,0.999883,0.022713,39.995208,...,0.101065,0.681054,0.203931,2.000115,0.100006,0.099334,9.936632,0.040073,1.020440,1.999023
6.950910,8,1.198872,0.758123,0.999844,0.010000,0.388248,4.591436,0.999995,0.024373,39.863482,...,0.100215,0.733839,0.227036,2.000083,0.100023,0.099570,9.912894,0.057912,1.020877,1.999747
6.953756,86,1.193425,0.685960,0.999851,0.010012,0.373503,4.163046,0.999983,0.023883,39.934227,...,0.100296,0.706250,0.204089,2.000086,0.100024,0.099628,9.999998,0.015715,1.020742,1.999993
6.953936,28,1.191406,0.687003,0.999280,0.010037,0.403160,4.130286,0.999697,0.024111,39.966909,...,0.102827,0.740777,0.234306,2.000356,0.100010,0.099718,9.995392,0.001011,1.021418,1.999984
6.954126,38,1.195641,0.721971,0.999962,0.010000,0.389960,4.353016,0.999689,0.023856,39.967650,...,0.100072,0.805132,0.247038,2.000601,0.100022,0.098702,9.998095,0.001002,1.022373,1.999832
6.956407,32,1.198452,0.745111,0.999415,0.010009,0.376124,4.460763,0.999982,0.023818,39.980590,...,0.100761,0.726073,0.210867,2.000027,0.100028,0.098927,9.986591,0.031148,1.020852,1.999833
6.956543,31,1.197107,0.663579,0.999251,0.010003,0.387143,3.966751,0.999805,0.023919,39.995189,...,0.106455,0.727347,0.232700,2.000705,0.100024,0.099621,9.996681,0.001000,1.023895,1.999929
6.957840,11,1.197506,0.696119,0.999817,0.010001,0.381130,4.182611,0.999990,0.024125,39.983515,...,0.108495,0.758768,0.250973,2.000051,0.100006,0.099375,9.962689,0.011072,1.021296,1.999813
6.961592,30,1.193790,0.736405,0.999859,0.010007,0.365906,4.451942,0.999913,0.024173,39.985942,...,0.100014,0.723347,0.213907,2.002373,0.100003,0.099968,9.893292,0.034762,1.022437,1.999990


In [ ]:
# Dosage:
DDR = 1

# Fixed parameters:
hcmdm21    = 2
hcwip11    = 2  #could be critical for getting quick peak dynamics for MCF7
#ksp210    = 0.00765773
#ksp211    = 0.788386
#jsp211    = 0.124622
#hcp211    = 2
#kshipk2    = [0.1,1]  #0.0659213  #
#kdp53      = [2,8]  #T12 = 5 to 20 min 1.3863 -> half life 30min -> 1.3 0.1 [0.01,1]1.99985  #
#kdp53a     = [0.1,0.2]  #half life 4h->0.17, 200min->0.2  0.1 [0.01,1]0.00116952  #
#kdp53k     = [0.1,0.2]  #0.0132642  #
kdhipk2    = 1.5  #HIPK2 protein degradation basal, half live 27min->1.5 stress induced up to >240 min depending on regulation  #
kdp53m     = 0.077  #p53mRNA half live 9h->0.077 0.243722  #
kdwip1m    = 0.66  #0.484087  #   #WIP1 mRNA degradation (half live ~1h, use kdeg 0.66 from doi.org/10.1016/j.cels.2016.03.006)  #
kdwip1     = 2  #0.776583  #   #WIP1 protein degradation, half live 20min ~1h -> kdeg=(2,0.7), doi.org/10.1016/j.molcel.2013.06.010 & 10.1083/jcb.201210031  #
kdmdm2m    = 0.7  #MDM2 mRNA deg, short half life < 1h->0.69   #
kdmdm2     = 2  #MDM2 protein deg half-live around 20min->2.08  #
#kdp21m     = 1.39   #0.620255  #
#kdbaxm     = 0.35   #1.40314  #
#jup53      = [0.01,0.1]  #0.05 MDM2_0=0.18 MCF7 data 0.00653985  #
#jup53s     = [0.1,1]  #0.5 0.120136  #
kuhipk2    = 0.17  #HIPK2 protein degradation, SIAH1 induced, greater T1/2 240min->0.17  #
#juhipk2    = [0.001,1]  #0.5 0.00951977  #
#ksbax0     = 0.225133
#ksbax1     = 0.439204
#jsbax1     = 0.00753637
#hcbax1     = 2
hcdeg      = 2
kalg       = 0.1  #0.109211  #

# Fixed ICs:
p_p53i_0 = 0.1       #from WB DR SY5Y
p_p53i_0_MCF7 = 0.056  #from WB TC MCF7
p_MDM2m_0_MCF7 = 0.18	#from MCF7 data at t=0
p_MDM2m_0 = 0.103		#0.18/1.75 because MDM2m 1.75 x higher in MCF7
p_p53m_0 = 0.1
p_p53m_0_MCF7 = 0.29	#2.9 fold change in expression//

WIP1m_0_MCF7 = 0.30  #from MCF7 data  #
WIP1_0_MCF7  = 0.14  #from MCF7 data#
WIP1m_0 = 0.0594		#WIP1 5.05 x higher in MCF7
WIP1_0  = 0.0277		#

MDM2_0 = 1			#
MDM2_0_MCF7 = 1.75 	#


# Initial conditions:
p_ATM_0 = 1
p_SIAH_0 = 1
p_HIPK2_0 = 0.018
p_CHK2 = 1
icZero =0
#p21m_0=0.1068
#BAXm_0=0.3

# Estimated parameters: loaded from bestFits file into param_est, param_id
try:
    i_selectedrun
except NameError:
    i_selectedrun = 0
print("Run " + str(i_selectedrun) + " selected. Cost function: " + str(est_params.index[i_selectedrun]))
#for i_init_pest=1:length(param_id)
#  (param_id{i_init_pest}) = param_est(i_selectedrun, i_init_pest)
#end

# Parameters fixed through algebraic constraints for IC  #
ksp530 = p_p53m_0*kdp53m  #rate of basal p53 mRNA expression determined by IC and kdeg  #
#--MDM2 and p53 --#
kpsmdm2 = MDM2_0*kdmdm2/p_MDM2m_0  #rate of MDM2 translation determined by ICs and kdeg  #
h_MDM2_0 = (MDM2_0**hcdeg)/(MDM2_0**hcdeg+est_params.iloc[i_selectedrun]['jup53']**hcdeg)
kpsp53 = est_params.iloc[i_selectedrun]['kdp53']*p_p53i_0/p_p53m_0*h_MDM2_0
h_p53i_0 = ((kalg*p_p53i_0)**hcmdm21)/(est_params.iloc[i_selectedrun]['jsmdm21']**hcmdm21+(kalg*p_p53i_0)**hcmdm21)
ksmdm20 = kdmdm2m*p_MDM2m_0-est_params.iloc[i_selectedrun]['ksmdm21']*h_p53i_0
#--WIP1--
kswip10 = kdwip1m*WIP1m_0
kpswip1 = kdwip1*WIP1_0/WIP1m_0
#--HIPK2--
p_kshipk2 = kdhipk2* p_HIPK2_0 + kuhipk2* p_SIAH_0*(p_HIPK2_0)/(est_params.iloc[i_selectedrun]['juhipk2']+p_HIPK2_0)
#--p21 and BAX--
#ksp210 = kdp21m*p21m_0
#ksbax0 = kdbaxm*BAXm_0

# make IC vector:
x0 = [p_ATM_0,icZero,p_p53m_0,p_p53i_0,icZero,icZero,p_SIAH_0,icZero,p_HIPK2_0,WIP1m_0,WIP1_0,p_CHK2,icZero,p_MDM2m_0,MDM2_0] # p21m_0, BAXm_0


Run 0 selected. Cost function: 6.942752


In [ ]:
# selecting patient specific parameters
NB_params = pd.read_csv('NB_params.csv')
#i_patientselected = 0

#NB_params.loc[i_patientselected,'ATM']


In [ ]:
def setpatientparams(i_patientselected):

    #Calculate patient-specific parameters:
    global kpsmdm2
    global kpsp53
    global kpswip1
    global x0
    global kshipk2

    #--ATM--
    ATM_0 = NB_params.loc[i_patientselected,'ATM']*p_ATM_0
    CHK2_0 = NB_params.loc[i_patientselected,'CHEK2']*p_CHK2
    #--p53--
    p53m = NB_params.loc[i_patientselected,'TP53']*p_p53m_0
    ksp530 = p53m*kdp53m  #rate of basal p53 mRNA expression determined by IC and kdeg  #
    #--MDM2--
    MDM2m = NB_params.loc[i_patientselected,'MDM2']*p_MDM2m_0
    MDM2 = MDM2m*kpsmdm2/kdmdm2  # MDM2 steady state
    h_MDM2 = (MDM2**hcdeg)/(MDM2**hcdeg+est_params.iloc[i_selectedrun]['jup53']**hcdeg)
    p53i = p53m*kpsp53/est_params.iloc[i_selectedrun]['kdp53']/h_MDM2
    h_p53i = ((kalg*p53i)**hcmdm21)/(est_params.iloc[i_selectedrun]['jsmdm21']**hcmdm21+(kalg*p53i)**hcmdm21)
    ksmdm20 = MDM2m*kdmdm2m-est_params.iloc[i_selectedrun]['ksmdm21']*h_p53i
    #--WIP1--
    WIP1m = NB_params.loc[i_patientselected,'PPM1D']*WIP1m_0
    kswip10 = WIP1m*kdwip1m
    WIP1 = WIP1m*kpswip1/kdwip1
    #--SIAH1--
    SIAH_0 = (1/2*(NB_params.loc[i_patientselected,'SIAH1']+NB_params.loc[i_patientselected,'WSB1']))*p_SIAH_0
    #--HIPK2--
    kshipk2 = NB_params.loc[i_patientselected,'HIPK2']*p_kshipk2
    # There are two ss solutions (you have to pick the positive one):
    # -(SIAH_0*kuhipk2 - kshipk2 + est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2 + (SIAH_0**2*kuhipk2**2 + 2*SIAH_0*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kuhipk2 - 2*SIAH_0*kshipk2*kuhipk2 + est_params.iloc[i_selectedrun]['juhipk2']**2*kdhipk2**2 + 2*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kshipk2 + kshipk2**2)**(1/2))/(2*kdhipk2)
    #  (kshipk2 - SIAH_0*kuhipk2 - est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2 + (SIAH_0**2*kuhipk2**2 + 2*SIAH_0*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kuhipk2 - 2*SIAH_0*kshipk2*kuhipk2 + est_params.iloc[i_selectedrun]['juhipk2']**2*kdhipk2**2 + 2*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kshipk2 + kshipk2**2)**(1/2))/(2*kdhipk2)
    HIPK2 = (kshipk2 - SIAH_0*kuhipk2 - est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2 + (SIAH_0**2*kuhipk2**2 + 2*SIAH_0*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kuhipk2 - 2*SIAH_0*kshipk2*kuhipk2 + est_params.iloc[i_selectedrun]['juhipk2']**2*kdhipk2**2 + 2*est_params.iloc[i_selectedrun]['juhipk2']*kdhipk2*kshipk2 + kshipk2**2)**(1/2))/(2*kdhipk2)
    #--p21 and BAX--
    # nothing to do, these will not be personalised. They were just included
    # for fitting the MCF7 data as proxi readout of p53s15 and p53s46.

    if (ksmdm20<0):
        print('Found negative patient-specific value for ksmdm20')
        ksmdm20 = 1e-9
    if (HIPK2<0):
        print('Found negative patient-specific value for HIPK2')
        HIPK2 = 1e-9

    # Parameters fixed through algebraic constraints for IC  #
    ksp530 = p_p53m_0*kdp53m  #rate of basal p53 mRNA expression determined by IC and kdeg  #
    #--MDM2 and p53 --#
    kpsmdm2 = MDM2_0*kdmdm2/p_MDM2m_0  #rate of MDM2 translation determined by ICs and kdeg  #
    h_MDM2_0 = (MDM2_0**hcdeg)/(MDM2_0**hcdeg+est_params.iloc[i_selectedrun]['jup53']**hcdeg)
    kpsp53 = est_params.iloc[i_selectedrun]['kdp53']*p_p53i_0/p_p53m_0*h_MDM2_0
    h_p53i_0 = ((kalg*p_p53i_0)**hcmdm21)/(est_params.iloc[i_selectedrun]['jsmdm21']**hcmdm21+(kalg*p_p53i_0)**hcmdm21)
    ksmdm20 = kdmdm2m*p_MDM2m_0-est_params.iloc[i_selectedrun]['ksmdm21']*h_p53i_0
    #--WIP1--
    kswip10 = kdwip1m*WIP1m_0
    kpswip1 = kdwip1*WIP1_0/WIP1m_0
    #--HIPK2--
    kshipk2 = kdhipk2* p_HIPK2_0 + kuhipk2* p_SIAH_0*(p_HIPK2_0)/(est_params.iloc[i_selectedrun]['juhipk2']+p_HIPK2_0)
    #--p21 and BAX--
    #ksp210 = kdp21m*p21m_0
    #ksbax0 = kdbaxm*BAXm_0

    # make IC vector:
    x0 = [ATM_0,icZero,p_p53m_0,p_p53i_0,icZero,icZero,SIAH_0,icZero,p_HIPK2_0,WIP1m_0,WIP1_0,p_CHK2,icZero,p_MDM2m_0,MDM2_0] # p21m_0, BAXm_0


## Defining ODE function

In [ ]:
def f(x,t):

    # SY5Y model:
    WIP1a = 1    # WIP1a = const.
    kpatm = est_params.iloc[i_selectedrun]['kpatm_dox']
    jpatm = est_params.iloc[i_selectedrun]['jpatm_dox']
    kdpatm = est_params.iloc[i_selectedrun]['kdpatm_dox']
    jdpatm = est_params.iloc[i_selectedrun]['jdpatm_dox']
    # MCF7 model: WIP1a = alpha*WIP1

    #States:
    ATM = x[0]
    ATMp = x[1]
    p53m = x[2]
    p53i = x[3]
    p53s15 = x[4]
    p53s46 = x[5]
    SIAH1 = x[6]
    SIAH1p = x[7]
    HIPK2 = x[8]
    WIP1m = x[9]
    WIP1 = x[10]
    CHK2 = x[11]
    CHK2p = x[12]
    MDM2m = x[13]
    MDM2 = x[14]

    #Input (e.g. step or constant)
    ku = np.heaviside(t,1)

    #Phosphorylation and dephosphorylation#
    v1=kpatm* ATM* DDR*(ku)/(jpatm+ATM)
    v2=kdpatm* WIP1a*(ATMp)/(jdpatm+ATMp)
    v3=est_params.iloc[i_selectedrun]['kpchk2']* ATMp*(CHK2)/(est_params.iloc[i_selectedrun]['jpchk2']+CHK2)
    v4=est_params.iloc[i_selectedrun]['kdpchk2']* WIP1*(CHK2p)/(est_params.iloc[i_selectedrun]['jdpchk2']+CHK2p)
    v5=est_params.iloc[i_selectedrun]['kpp53_CHK2']* CHK2p*(p53i)/(est_params.iloc[i_selectedrun]['jpp53_CHK2']+p53i)
    v6=est_params.iloc[i_selectedrun]['kpp53_ATM']* ATMp*(p53i)/(est_params.iloc[i_selectedrun]['jpp53_ATM']+p53i)
    v7=est_params.iloc[i_selectedrun]['kdpp53a']* WIP1*(p53s15)/(est_params.iloc[i_selectedrun]['jdpp53a']+p53s15)
    v8=est_params.iloc[i_selectedrun]['kpp53a']* HIPK2*(p53s15)/(est_params.iloc[i_selectedrun]['jpp53a']+p53s15)
    v9=est_params.iloc[i_selectedrun]['kdpp53k']*(p53s46)/(est_params.iloc[i_selectedrun]['jdpp53k']+p53s46)
    v10=est_params.iloc[i_selectedrun]['kpsiah1']* ATMp*(SIAH1)/(est_params.iloc[i_selectedrun]['jpsiah1']+SIAH1)
    v11=est_params.iloc[i_selectedrun]['kdpsiah1']*(SIAH1p)/(est_params.iloc[i_selectedrun]['jdpsiah1']+SIAH1p)

    #Gene-expression: mRNA synthesis#
    # we need p53 activity and p53s15 total
    p53alg=kalg*p53i+p53s15+p53s46
    p53s15tot=p53s15+p53s46
    v12=ksp530
    v13=ksmdm20
    v14=est_params.iloc[i_selectedrun]['ksmdm21']*(p53alg**hcmdm21)/(est_params.iloc[i_selectedrun]['jsmdm21']**hcmdm21+p53alg**hcmdm21)
    v15=kswip10
    v16=est_params.iloc[i_selectedrun]['kswip11']*(p53s15tot**hcwip11)/(est_params.iloc[i_selectedrun]['jswip11']**hcwip11+p53s15tot**hcwip11)
    #v17=ksp210
    #v18=ksp211*(p53s15**hcp211)/(jsp211**hcp211+p53s15**hcp211)
    #v19=ksbax0
    #v20=ksbax1*(p53s46**hcbax1)/(jsbax1**hcbax1+p53s46**hcbax1)

    #Translation: protein synthesis#
    v21=kpsp53*p53m
    v22=kshipk2
    v23=kpswip1*WIP1m
    v24=kpsmdm2*MDM2m

    #Degradation of mRNAs and proteins#
    v25=est_params.iloc[i_selectedrun]['kdp53']* p53i*(MDM2**hcdeg)/(MDM2**hcdeg+est_params.iloc[i_selectedrun]['jup53']**hcdeg)
    v26=est_params.iloc[i_selectedrun]['kdp53a']* p53s15*(MDM2**hcdeg)/(MDM2**hcdeg+est_params.iloc[i_selectedrun]['jup53s']**hcdeg)
    v27=est_params.iloc[i_selectedrun]['kdp53a']* p53s46*(MDM2**hcdeg)/(MDM2**hcdeg+est_params.iloc[i_selectedrun]['jup53s']**hcdeg)
    v28=kdhipk2* HIPK2
    v29=kdp53m* p53m
    v30=kdwip1m* WIP1m
    v31=kdwip1* WIP1
    v32=kdmdm2m* MDM2m
    v33=kdmdm2* MDM2
    #v34=kdp21m* p21m
    #v35=kdbaxm* BAXm
    v36=kuhipk2* SIAH1*(HIPK2)/(est_params.iloc[i_selectedrun]['juhipk2']+HIPK2)

    p53tot=p53i+p53s15+p53s46

    #System of ordinary differential equations of the p53 model

    ATM_d=-v1+v2
    ATMp_d=v1-v2
    p53m_d=v12-v29
    p53i_d=-v5-v6+v7+v21-v25
    p53s15_d=v5+v6-v7-v8+v9-v26
    p53s46_d=v8-v9-v27
    SIAH1_d=-v10+v11
    SIAH1p_d=v10-v11
    HIPK2_d=v22-v28-v36
    WIP1m_d=v15+v16-v30
    WIP1_d=v23-v31
    CHK2_d=-v3+v4
    CHK2p_d=v3-v4
    MDM2m_d=v13+v14-v32
    MDM2_d=v24-v33
    #p21m_d=v17+v18-v34
    #BAXm_d=v19+v20-v35

    return [ATM_d,ATMp_d,p53m_d,p53i_d,p53s15_d,p53s46_d,SIAH1_d,SIAH1p_d,HIPK2_d,WIP1m_d,WIP1_d,CHK2_d,CHK2p_d,MDM2m_d,MDM2_d] #  p21m_d,BAXm_d


## Solving ODE system

In [ ]:
DDRvec = np.logspace(1e-3,10,num=10)

Tstart = 0
Tfinish = 24
numberOfPoints = 20
tspan = np.linspace(Tstart, Tfinish, num=numberOfPoints)

plt.figure()

for i in range(2):
    setpatientparams(i)
    p53s15DR = []
    p53s46DR = []
    for DDRval in DDRvec:
        DDR = DDRval
        cons = scipy.integrate.odeint(f,x0,tspan)
        p53s15DR.append(cons[-1,4])
        p53s46DR.append(cons[-1,5])
    plt.plot(DDRvec,p53s46DR)

plt.xscale('log')
plt.show()